In [1]:
import pandas as pd
pd.set_option('display.max_columns', 600)

from scipy import stats
import numpy as np

In [2]:
rna = pd.read_csv("./cBioPotal/all_phase2_target_2018_pub_(pediatric)/data_RNA_Seq_expression_median.txt", sep = '\t', header = None)
table = pd.read_csv("./cBioPotal/all_phase2_target_2018_pub_(pediatric)/all_phase2_target_2018_pub_clinical_data.csv")

#RNA preprocessing
rna.columns = rna.iloc[0,:]
del rna['Entrez_Gene_Id']
rna = rna.drop_duplicates('Hugo_Symbol', keep = 'first').dropna()

rna = rna.T

rna.columns = rna.iloc[0,:]
rna = rna[1:]

table4 = pd.merge(table, rna, left_on = 'Sample ID', right_on = 'Hugo_Symbol')

C:\Users\bioinfo205_01\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
#########################################non-paired sample
m = table4['Sample ID'].str.contains('-04') == True
m2 = table4['Sample ID'].str.contains('-09') == True

paired = pd.concat([table4[m], table4[m2]])
paired = paired.sort_values(by='Sample ID').reset_index()

paired['Relapse'] = paired['Sample ID'].str.contains('-04') == True

print(paired.shape)

#data output
eset = paired.loc[:,'TSPAN6':'LIN37']
pset = paired.loc[:, ['Sample ID', 'Relapse']]

eset.to_csv('./eset.csv')
pset.to_csv('./pset.csv')

(163, 22899)


In [60]:
########################################paired sample
df = table4[table4['Sample ID'].str.contains('-04')]
df2 = table4[table4['Sample ID'].str.contains('-09')]

cols = list(df.columns)

df3 = pd.merge(df, df2, on='Patient ID')

df = df3.loc[:,:'LIN37_x']
df2 = df3.loc[:,'Study ID_y':]
df2['Patient ID'] = df3['Patient ID']

df.columns = cols

cols.remove('Patient ID')
cols.append('Patient ID')

df2.columns = cols

paired = pd.concat([df, df2], axis=0)

paired = paired.sort_values(by='Sample ID')
paired['Relapse'] = paired['Sample ID'].str.contains('-04') == True

pcols = list(rna.columns)
pcols.remove('Hugo_Symbol')

print(paired.shape)

#data output
eset = paired[pcols].reset_index(drop=True)
pset = paired.loc[:, ['Sample ID', 'Relapse']].reset_index(drop=True)

eset.to_csv('./eset.csv')
pset.to_csv('./pset.csv')

C:\Users\bioinfo205_01\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(78, 22898)


In [54]:
###########################################DEG processing
target_fdr = 0.05
target_logfc = 1.5

#deg cutoff
deg = pd.read_csv('./res.csv')

def degpl(deg):
    m = deg['adj.P.Val'] < target_fdr
    m_pl = deg['logFC'] > target_logfc
    deg_pl = deg[m][m_pl]
    
    print('deg_pl: ', deg_pl.shape)
    return deg_pl

def degmi(deg):
    m = deg['adj.P.Val'] < target_fdr
    m_mi = deg['logFC'] < -target_logfc
    
    deg_mi = deg[m][m_mi]
    print('deg_mi: ', deg_mi.shape)
    return deg_mi

def changechr(n):
    try:
        return n.split('.')[0] + '-' + n.split('.')[1]
    except:
        return n

print('deg----------------------------')
deg_pl = degpl(deg)
deg_mi = degmi(deg)
deg_pl["Unnamed: 0"] = deg_pl["Unnamed: 0"].apply(lambda x: changechr(x))
deg_mi["Unnamed: 0"] = deg_mi["Unnamed: 0"].apply(lambda x: changechr(x))

deg_pl.to_csv('./deg_pl.csv')
deg_mi.to_csv('./deg_mi.csv')

deg----------------------------
deg_pl:  (0, 7)
deg_mi:  (0, 7)


C:\Users\bioinfo205_01\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
C:\Users\bioinfo205_01\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
